In [ ]:
import random

class Item: #small and large items to be obtained (hold 2 large, any number of small)
    def __init__(self, name, size = 'small', item_type = None, effect = None, heal = 0, attack = 0, defense = 0):
        self.name = name
        self.size = size 
        self.item_type = item_type
        self.effect = effect
        self.heal = heal
        self.attack = attack
        self.defense = defense

    def use(self, character):
        if self.item_type == 'weapon':
            character.attack += self.effect
            print(f"You equip {self.name}. Your attack is now {character.attack}.") 
        elif self.item_type == 'shield':
            character.defense += self.effect
            print(f"You equip {self.name}. Your defense is now {character.defense}.")
        elif self.item_type == 'heal':
            character.heal(self.effect)
            print(f"You use {self.name} and heal for {self.heal}. Your health is now {character.health}.")  

class Character_Inventory: #managable inventory system to add/remove gained items
    def __init__(self):
        self.small_items = [] 
        self.large_items = []
    
    def add_item(self, item):
        if item.size == 'large' and len(self.large_items) < 2:
            self.large_items.append(item)
            print(f"You pick up {item.name}. It looks too heavy for the bag, you'll have to put it in your hand.")
        elif item.size == 'small':
            self.small_items.append(item)
            print(f"You pick up {item.name} and add it to your bag.")
        else:
            print("You can can only carry two large items. You've only got two hands after all.")
    
    def remove_item(self, item):
        if item in self.large_items:
            self.large_items.remove(item)
            print(f"You drop the {item.name}.")
        elif item in self.small_items:
            self.small_items.remove(item)
            print(f"You drop the {item.name}.")
        else:
            print(f"{item.name} is not in your inventory.")
    
    def view_inventory(self): 
        print("You peer into your bag, finding: ")
        for item in self.small_items:
            print(f" - {item.name} (Small) | Heal: {item.heal}, Attack: {item.attack}, Defense: {item.defense}")
        for item in self.large_items:
            print(f" - {item.name} (Large) | Heal: {item.heal}, Attack: {item.attack}, Defense: {item.defense}")

small_loot_pool = [
    ("Bandages", 'heal', 10, 0, 0),
    ("Syringe", 'heal', 7, random.randint(2, 5), 0),
    ("Experimental Ammunition", 'weapon', 0, random.randint(1, 3), 0),
    ("Duct Tape", 'heal', 3, 0, 0),
    ("MedKit", 'heal', 15, 0, 0),
    ("Wrappings", 'heal', 2, 0, 1),
]

large_loot_pool = [
    ("Riot Shield", 'shield', 0, 0, 7),
    ("Repair Kit", 'heal', random.randint(15, 30), 0, 0),
    ("Assault Rifle", 'weapon', 0, random.randint(7, 10), 0),
    ("Flamethrower", 'weapon', 0, random.randint(5, 7), 0),
    ("Broken Pipe", 'weapon', 0, random.randint(3, 6), 0),
    ("Wall Panel", 'shield, 0, 0, 3'),
    ("Fire Extinguisher", 'weapon', 0, random.randint(2, 4), 0),
    ("Corpse", 'shield', 0, 0, random.randint(4, 5)),
]

class Room: #create rooms and keep track of whether they've been visited
    global current_room_index

    def __init__(self, room_id, room_type = None):
        self.room_id = room_id
        self.room_type = room_type
        self.visited = False
        self.effect = None
        self.small_loot_count = 0
        self.large_loot_count = 0
        self.enemy = None

    def apply_random_effect(self, character, rooms):
        effects = ["hazardous waste", "burning", "stim_shot", "enemy_encounter", "nothing"]
        chosen_effect = random.choice(effects)

        if chosen_effect == "hazardous waste":
            hazardous_waste = random.randint(5, 15)
            character.health -= hazardous_waste
            print(f"You step in spilled waste. You take {hazardous_waste} damage and are now at {character.health} health.")
        elif chosen_effect == "burning":
            burning = random.randint(3, 7)
            character.health -= burning
            print(f"As you walk into the room, you begin to choke on the smoke that billows out from it. Ever fearless, you push through the flames that have engulfed the room. \nYou take {burning} damage and are now at {character.health} health.")
        elif chosen_effect == "stim_shot":
            stim_shot = random.randint(6, 10)
            character.health += stim_shot
            print(f"As you enter the room, you begin to feel rejuvenated. \nA surge of energy washes over you and you realize you are now at {character.health} health.")
        elif chosen_effect == "enemy_encounter":
            encounter_enemy(character, rooms)
        elif chosen_effect == "nothing":
            print("You enter the room and feel... nothing? This seems like a safe place to get your bearings.")

    def visit(self, character, rooms):
        if not self.visited:
            self.visited = True
            self.apply_random_effect(character, rooms)
            self.generate_loot()
            if self.room_type == 'left_generator_room':
                self.fix_generator(character, 'left')
            elif self.room_type == 'right_generator_room':
                self.fix_generator(character, 'right')
            elif self.room_type == 'reactor_room':
                self.start_reactor_meltdown(character)
    
    def search_room(self, character):
        if self.small_loot_count > 0 or self.large_loot_count > 0:
            print(f"You search the room...")

            for _ in range(self.small_loot_count):
                loot_item_data = random.choice(small_loot_pool)
                loot_item = Item(loot_item_data[0], 'small', loot_item_data[1], 
                                 effect=loot_item_data[2], heal=loot_item_data[2], 
                                 attack=loot_item_data[3], defense=loot_item_data[4])
                character.inventory.add_item(loot_item)

            for _ in range(self.large_loot_count):
                loot_item_data = random.choice(large_loot_pool)
                loot_item = Item(loot_item_data[0], 'large', loot_item_data[1], 
                                 effect=loot_item_data[2], heal=loot_item_data[2], 
                                 attack=loot_item_data[3], defense=loot_item_data[4])
                character.inventory.add_item(loot_item)
            
            self.small_loot_count = 0
            self.large_loot_count = 0

        else:
            print("You do not find anything in the room.")

    def generate_loot(self):
        self.small_loot_count = random.randint(1, 3) 
        self.large_loot_count = random.randint(0, 1) 

    def fix_generator(self, character, side):
        if side == 'left':
            character.fix_left_generator()
        elif side == 'right':
            character.fix_right_generator()

    def start_reactor_meltdown(self, character):
        character.start_reactor_meltdown()

def create_rooms():
    room_types = ['left_generator_room', 'right_generator_room', 'reactor_room', 'greenroom', 'laboratory wing', 'security office', 'living quarters', 'emergency room', 'storage', 'canteen', 'boardroom', 'shower room', 'armory']
    rooms = []
    
    for i in range(1, 14):
        room_type = random.choice(room_types)
        room_types.remove(room_type) if room_type in ['left_generator_room', 'right_generator_room', 'reactor_room', 'greenroom', 'laboratory wing', 'security office', 'living quarters', 'emergency room', 'storage', 'canteen', 'boardroom', 'shower room', 'armory'] else None
        room = Room(i, room_type)
        rooms.append(room)

    random.shuffle(rooms) 
    
    return rooms
current_room_index = 0 #start from first room

class Character_Class: #initialize character classes/weapons/stats
    def __init__(self, name, weapon, health, attack, defense):
        self.name = name
        self.weapon = weapon
        self.inventory = Character_Inventory()
        self.health = health
        self.attack = attack
        self.defense = defense
        self.left_generator_fixed = False  
        self.right_generator_fixed = False 
        self.reactor_meltdown_started = False
        self.equipped_large_items = []

    def fix_left_generator(self):
        if not self.left_generator_fixed:
            self.left_generator_fixed = True
            print("Left generator has been repaired.")
        else:
            print("Left generator is already repaired.")
    
    def fix_right_generator(self):
        if not self.right_generator_fixed:
            self.right_generator_fixed = True
            print("Right generator has been repaired.")
        else:
            print("Right generator is already repaired.")
    
    def start_reactor_meltdown(self):
        if self.left_generator_fixed and self.right_generator_fixed:
            self.reactor_meltdown_started = True
            print("Countdown begun. We appreciate your service to the company.")
        else:
            print("You must repair both generators before you can start the meltdown sequence.")
    
    def use_item(self, item):
        if item.item_type == "heal":
            item.use(self)
            if item in self.inventory.small_items:
                self.inventory.remove_item(item)
        elif item.item_type == "weapon":
            self.apply_weapon(item)
            if item in self.inventory.small_items:
                self.inventory.remove_item(item)
        elif item.item_type == "shield":
            self.apply_shield(item)
            if item in self.inventory.small_items:
                self.inventory.remove_item(item)
        else:
            print(f"You are unable to use {item.name} at this time.")
    
    def apply_weapon(self, weapon=None):
        if weapon:
            self.attack += weapon.effect
            print(f"Weapon equipped: {weapon.name} Attack is now {self.attack}.")
        else:
            print(f"You decide to continue with your current weapon. Attack is {self.attack}.")

    def apply_shield(self, shield=None):
        if shield:
            self.defense += shield.effect
            print(f"You raise your {shield.name}. Defense is now {self.defense}.")
        else:
            print(f"Your arm lays bare. Defense is {self.defense}.")
    
    def equip_large_item(self, item):
        if item.size == "large":
            self.equipped_large_items.append(item)
            self.attack += item.attack
            self.defense += item.defense
            print(f"You equip {item.name}. Your attack is now {self.attack} and defense is {self.defense}.")

    def unequip_large_item(self, item):
        if item in self.equipped_large_items:
            self.equipped_large_items.remove(item)
            self.attack -= item.attack
            self.defense -= item.defense
            print(f"You unequip {item.name}. Your attack is now {self.attack} and defense is {self.defense}.")

    def takes_damage(self, damage):
        damage_taken = max(0, damage - self.defense)
        self.health -= damage_taken
        if self.health < 0:
            self.health = 0
        print(f"You are hit for {damage_taken} and have {self.health} health remaining.")

    def lives(self):
        return self.health > 0
    
    def heal(self, healing_amount):
        self.health += healing_amount
        print(f"You heal {healing_amount} and now have {self.health} remaining.")
    
    def update_stats(self):
        self.attack = 10
        self.defense = 5 
        self.apply_weapon()
        self.apply_shield()

class Character_Creator:
    def __init__(self):
        self.classes = {
            1: {"name": "Engineer", "weapon": "Plasma Cutter", "health": 120, "attack": 8, "defense": 4},
            2: {"name": "Overseer", "weapon": "Pistol", "health": 100, "attack": 12, "defense": 5},
            3: {"name": "Scientist", "weapon": "Experimental Rifle", "health": 80, "attack": 16, "defense": 3},
            4: {"name": "Security Guard", "weapon": "Assault Rifle", "health": 150, "attack": 10, "defense": 7}
        }

    def create_character(self):
        print("Choose your character class:")
        for class_id, class_info in self.classes.items():
            print(f"{class_id}. {class_info['name']}")

        while True:
            try:
                class_choice = int(input("Enter the number of your chosen class: "))
                if class_choice in self.classes:
                    chosen_class = self.classes[class_choice]
                    character = Character_Class(
                        name=input("Enter your character's name: "),
                        weapon=chosen_class["weapon"],
                        health=chosen_class["health"],
                        attack=chosen_class["attack"],
                        defense=chosen_class["defense"],
                    )
                    print(f"You have chosen the {chosen_class['name']} class.")
                    return character
                else:
                    print("The company does not hire that profession. Please choose from our existing list.")
            except ValueError:
                print("Please enter a number corresponding to the class.")
        
class Enemy:
    def __init__(self, name, health, attack, defense):
        self.name = name
        self.health = health
        self.attack = attack
        self.defense = defense
    
    def takes_damage(self, damage):
        damage_taken = max(0, damage - self.defense)
        self.health -= damage_taken
        if self.health < 0:
            self.health = 0
        print(f"{self.name} is hit for {damage_taken} and has {self.health} health remaining.")
        
    def attacks_player(self, character):
        damage_dealt = max(0, self.attack - character.defense)
        character.health -= damage_dealt
        if character.health < 0:
            character.health = 0
        print(f"{self.name} swings for {damage_dealt} damage. You have {character.health} health left.")
    
    def lives(self):
        return self.health > 0

def encounter_enemy(character, rooms):
    global current_room_index
    enemy = Enemy("Specimen", health = random.randint(13, 70), attack = random.randint(10, 20), defense = random.randint(4, 12))
    print(f"A {enemy.name} stares back at you from across the room.")

    while enemy.lives() and character.lives():
        print(f"\n{character.name} - {character.health} health left. \n{enemy.name} - {enemy.health} health left.")
        action_choice = input("\nChoose an action: \n1. Attack \n2. Use Item \n3. Run")
        try:
            if action_choice == "1": #attack
                print(f"{character.name} attacks {enemy.name}.")
                enemy.takes_damage(character.attack)
                if enemy.lives():
                    enemy.attacks_player(character)
            elif action_choice == "2": #Use Item
                print("Choose an item to use.")
                if character.inventory.small_items:
                    print("Your items: ")
                    for idx, item in enumerate(character.inventory.small_items, 1):
                        print(f"{idx}. {item.name}")

                    item_choice = input("Choose an item to use: ")
                    try: 
                        item_idx = int(item_choice) - 1
                        if 0 <= item_idx < len(character.inventory.small_items):
                            item = character.inventory.small_items[item_idx]
                            if item.item_type == "heal":
                                item.use(character)
                                character.inventory.remove_item(item)
                            else:
                                print(f"{item.name} is not usable right now.")
                        else:
                            print("You do not own this item.")
                    except ValueError:
                        print("Input not recognized. Try again.")
                else:
                    print("Your bag is empty.")
                if enemy.lives():
                    enemy.attacks_player(character)
            elif action_choice == "3": #run
                print(f"You attempt to sidestep the {enemy.name} and rush to the next room.")
                escape_chance = random.randint(1, 10)
                success_chance = 7

                if escape_chance >= success_chance:
                    print("Your gambit is successful, you successfully make it to the next room.")
                    movement = input("Would you like to move forward or backward? (1/2)")
                    if movement == '1':
                        if current_room_index < len(rooms) - 1:
                            current_room_index += 1
                            print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                        else:
                            print("You are at the last room. You can only go backward.")
                    elif movement == '2':
                        if current_room_index > 0:
                            current_room_index -= 1
                            print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                        else:
                            print("You are currently in the first room. You can only go forward.")
                    else:
                        print("Please choose between forward and backward.")
                else:
                    print(f"The {enemy.name} blocks your exit.")
                    enemy.attacks_player(character)
        except ValueError:
            print("Please choose a valid action.")
            
        if not enemy.lives():
                print(f"You stand over the {enemy.name}'s corpse")
                break
        elif not character.lives():
                print(f"Rest in Peace to {character.name}. Here's hoping their successor learns from their mistakes.")
                return

def explore_room(character, rooms): #room exploration and turn taking
    global current_room_index

    while True: #action loop
        print(f"\nYou are currently in Room {rooms[current_room_index].room_id}.")
        rooms[current_room_index].visit(character, rooms)

        if character.reactor_meltdown_started:
            return
        
        print("\nChoose your action:")
        print("1. Search the room")
        print("2. Move to the next room")
        print("3. Move Backward")
        print("4. Check Inventory (NO ACTION USED)")
        print("5. Quit Game")


        action_choice = input("What would you like to do? (1/2/3/4/5/6): ").strip()

        if action_choice == '1': #search room
            rooms[current_room_index].search_room(character)
            print(f"\nroom searched")
        elif action_choice == '2': #move forward
            if current_room_index < len(rooms) - 1:
                current_room_index += 1
                print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
            else:
                print("You are at the last room. You can only go backward.")
        elif action_choice == '3': #move backward
            if current_room_index > 0:
                current_room_index -= 1
                print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
            else:
                print("You are currently in the first room. You can only go forward.")
        elif action_choice == '4': #view inventory
            character.inventory.view_inventory()
            use_item_choice = input("\nWould you like to use anything? (y/n): ").strip().lower()
            if use_item_choice == 'y' or use_item_choice == 'yes':
                print("Choose an item to use.")
                if character.inventory.small_items or character.inventory.large_items:
                    print("Your inventory: ")
                    if character.inventory.small_items:
                        print("Small Items:")
                        for idx, item in enumerate(character.inventory.small_items, 1):
                            print(f"{idx}. {item.name}")
                    if character.inventory.large_items:
                        print("Large Items:")
                        for idx, item in enumerate(character.inventory.large_items, 1):
                            print(f"{idx}. {item.name}")
                    
                    item_choice = input("Choose an item number to use (or press Enter to cancel): ").strip()

                    if item_choice: 
                        try:
                            item_idx = int(item_choice) - 1
                            if 0 <= item_idx < len(character.inventory.small_items):
                                item = character.inventory.small_items[item_idx]
                                character.use_item(item) 
                                character.inventory.remove_item(item)  
                            elif 0 <= item_idx < len(character.inventory.large_items):
                                item = character.inventory.large_items[item_idx]
                                character.use_item(item)  
                                character.inventory.remove_item(item) 
                            else:
                                print("Invalid choice, please select a valid item.")
                        except ValueError:
                            print("Invalid input, please enter a number.")
                else:
                    print("Your inventory is empty. Nothing to use.")
            else:
                print("You decide not to use an item.")
        elif action_choice == '5': #break loop and end game
            print("You have abandoned the company.")
            return
        else:
            print("Invalid action. Please choose again.")
        
def class_pick():
    global current_room_index
    rooms = create_rooms()

    creator = Character_Creator()
    character = creator.create_character()
    final_choice = input(f"So you are {character.name}? (y/n) ").lower()

    if final_choice == 'y' or final_choice == 'yes':
        print(f"Welcome {character.name}! \nPlease take the provided {character.weapon}.") 
        print("The company has been authorized to offer additional support should you seek it, but they require additional tasks to be completed in return.")
        print("The tasks are as follows: ")
        
        explore_room(character, rooms) 
       
    else: 
        print("\nI'm sorry, you don't seem to be in our database.")
        try_again = input("Would you like to try again?: ").lower()

        if try_again != 'y' and try_again != 'yes':
            class_pick()
        else:
            print("Server Locking.")
           
class_pick()